In [1]:
from calitp.tables import tbls

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(20_000_000_000)

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import calitp.magics

In [3]:
from calitp import query_sql

In [4]:
SELECTED_DATE = "2022-11-08"
SELECTED_HOUR = 0
ITP_ID_URL_NUM_LIST = [
    [4,1],
    [10,1],
    [279,1],
    [246,1],
    [56,1],
    [76,1],
    [61,1],
    [106,0],
    [110,1],
    [127,1],
    [194,0],
    [282,0],
    [217,0],
    [247,1],
    [264,1],
    [290,1],
    [280,0],
    [294,1],
    [310,1],
    [314,1],
    [315,1],
    [338,1],
    [336,0],
    [167,1],
    [350,0],
    [356,1],
    [218,1],
    [368,1],
    [98,0],
    [281,0],
    [301,0],
    [15,0],
    [481,0],
    [485,0]
    
]

In [5]:
# Build out query of ITP IDs and URL Numbers for 511 feeds
ITP_CLAUSE = "("
for id_url_combo in ITP_ID_URL_NUM_LIST:
    # check if on second+ element
    if ITP_CLAUSE != '(':
        # this is on the second or more clause, so we need to add an "OR" statement
        ITP_CLAUSE += ' OR '
        
    # main body of the subclause of ITP ID and URL Number combos
    ITP_CLAUSE += '(itp_id = ' + str(id_url_combo[0]) + ' and url_number = ' + str(id_url_combo[1]) + ')'

# Wrap up subclause with closing parenthesis    
ITP_CLAUSE += ')'

# Create overall query for vehicle positions
RT_VEHICLES_QUERY = """
SELECT
    itp_id,
    url_number,
    count(vehicle)
from cal-itp-data-infra.external_gtfs_rt.vehicle_positions
where dt = '""" + SELECTED_DATE + """'
and hour = """ + str(SELECTED_HOUR) + """
and """ + ITP_CLAUSE + """
group by itp_id,url_number
LIMIT 100"""

# print overall query
RT_VEHICLES_QUERY

"\nSELECT\n    itp_id,\n    url_number,\n    count(vehicle)\nfrom cal-itp-data-infra.external_gtfs_rt.vehicle_positions\nwhere dt = '2022-11-08'\nand hour = 0\nand ((itp_id = 4 and url_number = 1) OR (itp_id = 10 and url_number = 1) OR (itp_id = 279 and url_number = 1) OR (itp_id = 246 and url_number = 1) OR (itp_id = 56 and url_number = 1) OR (itp_id = 76 and url_number = 1) OR (itp_id = 61 and url_number = 1) OR (itp_id = 106 and url_number = 0) OR (itp_id = 110 and url_number = 1) OR (itp_id = 127 and url_number = 1) OR (itp_id = 194 and url_number = 0) OR (itp_id = 282 and url_number = 0) OR (itp_id = 217 and url_number = 0) OR (itp_id = 247 and url_number = 1) OR (itp_id = 264 and url_number = 1) OR (itp_id = 290 and url_number = 1) OR (itp_id = 280 and url_number = 0) OR (itp_id = 294 and url_number = 1) OR (itp_id = 310 and url_number = 1) OR (itp_id = 314 and url_number = 1) OR (itp_id = 315 and url_number = 1) OR (itp_id = 338 and url_number = 1) OR (itp_id = 336 and url_numbe

In [6]:
df_vehicles = query_sql(RT_VEHICLES_QUERY, as_df=True)

In [7]:
# convert dataframe to list because I, Evan, am not a pandas whiz
itp_ids_with_rt = df_vehicles['itp_id'].tolist()

# create placeholder list for itp_ids without RT
itp_ids_without_rt_vehicles = []

# iterate through input list to check if RT was found to exist
for itp_id in ITP_ID_URL_NUM_LIST:
    cur_id = itp_id[0]
    if cur_id not in itp_ids_with_rt:
        itp_ids_without_rt_vehicles.append(cur_id)
        
# print out ITP_IDs with no data
print('ITP IDs with no RT vehicle data')
print(sorted(itp_ids_without_rt_vehicles))

ITP IDs with no RT vehicle data
[10, 15, 56, 76, 264, 279, 281, 282, 338, 356, 368, 481, 485]


In [8]:
# Create overall query for trip updates
RT_TRIPS_QUERY = """
SELECT
    itp_id,
    url_number,
    count(tripUpdate)
FROM `cal-itp-data-infra.external_gtfs_rt.trip_updates`
where dt = '""" + SELECTED_DATE + """'
and hour = """ + str(SELECTED_HOUR) + """
and """ + ITP_CLAUSE + """
group by itp_id,url_number
LIMIT 100"""

# print overall query
RT_TRIPS_QUERY

"\nSELECT\n    itp_id,\n    url_number,\n    count(tripUpdate)\nFROM `cal-itp-data-infra.external_gtfs_rt.trip_updates`\nwhere dt = '2022-11-08'\nand hour = 0\nand ((itp_id = 4 and url_number = 1) OR (itp_id = 10 and url_number = 1) OR (itp_id = 279 and url_number = 1) OR (itp_id = 246 and url_number = 1) OR (itp_id = 56 and url_number = 1) OR (itp_id = 76 and url_number = 1) OR (itp_id = 61 and url_number = 1) OR (itp_id = 106 and url_number = 0) OR (itp_id = 110 and url_number = 1) OR (itp_id = 127 and url_number = 1) OR (itp_id = 194 and url_number = 0) OR (itp_id = 282 and url_number = 0) OR (itp_id = 217 and url_number = 0) OR (itp_id = 247 and url_number = 1) OR (itp_id = 264 and url_number = 1) OR (itp_id = 290 and url_number = 1) OR (itp_id = 280 and url_number = 0) OR (itp_id = 294 and url_number = 1) OR (itp_id = 310 and url_number = 1) OR (itp_id = 314 and url_number = 1) OR (itp_id = 315 and url_number = 1) OR (itp_id = 338 and url_number = 1) OR (itp_id = 336 and url_numbe

In [11]:
# TODO: this exceeds the query limit. 
# Might need to loop over each feed and make an individual query
df_trips = query_sql(RT_TRIPS_QUERY, as_df=True)

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 500 Query exceeded limit for bytes billed: 5000000000. 5860491264 or higher required.

Location: us-west2
Job ID: fdb12601-c582-4331-925d-572ec7d50123

[SQL: 
SELECT
    itp_id,
    url_number,
    count(tripUpdate)
FROM `cal-itp-data-infra.external_gtfs_rt.trip_updates`
where dt = '2022-11-08'
and hour = 0
and ((itp_id = 4 and url_number = 1) OR (itp_id = 10 and url_number = 1) OR (itp_id = 279 and url_number = 1) OR (itp_id = 246 and url_number = 1) OR (itp_id = 56 and url_number = 1) OR (itp_id = 76 and url_number = 1) OR (itp_id = 61 and url_number = 1) OR (itp_id = 106 and url_number = 0) OR (itp_id = 110 and url_number = 1) OR (itp_id = 127 and url_number = 1) OR (itp_id = 194 and url_number = 0) OR (itp_id = 282 and url_number = 0) OR (itp_id = 217 and url_number = 0) OR (itp_id = 247 and url_number = 1) OR (itp_id = 264 and url_number = 1) OR (itp_id = 290 and url_number = 1) OR (itp_id = 280 and url_number =

In [12]:
# convert dataframe to list because I, Evan, am not a pandas whiz
itp_ids_with_rt = df_trips['itp_id'].tolist()

# create placeholder list for itp_ids without RT
itp_ids_without_rt_trips = []

# iterate through input list to check if RT was found to exist
for itp_id in ITP_ID_URL_NUM_LIST:
    cur_id = itp_id[0]
    if cur_id not in itp_ids_with_rt:
        itp_ids_without_rt_trips.append(cur_id)
        
# print out ITP_IDs with no data
print('ITP IDs with no RT trip data')
print(sorted(itp_ids_without_rt_trips))

ITP IDs with no RT trip data
[10, 15, 56, 76, 264, 279, 281, 282, 338, 356, 368, 481, 485]
